In [43]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

import matplotlib.pyplot as plt

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

In [44]:
import brightway2 as bw

In [3]:
bw.projects.set_current('non-linearity')

In [45]:
bw.projects.set_current("remind_36")
bw.databases

Databases dictionary with 22 objects, including:
	biosphere3
	cutoff35
	ecoinvent 3.6 cutoff
	ecoinvent_SSP2-Base_2015
	ecoinvent_SSP2-Base_2028
	ecoinvent_SSP2-Base_2030
	ecoinvent_SSP2-Base_2040
	ecoinvent_SSP2-Base_2045
	ecoinvent_SSP2_Baseline_newIndustry_2015
	ecoinvent_SSP2_Baseline_newIndustry_2020
Use `list(this object)` to get the complete list.

In [46]:
mydb = bw.Database('ecoinvent 3.6 conseq_q')
mydb = bw.Database('ecoinvent 3.6 cutoff')

In [47]:
#to select only transforming activities later
transf_act = [act for act in mydb 
                    if 'ordinary transforming activity' in act['activity type'] 
             
     ]

In [48]:
FU = [{mydb.random():1} for act in range(50)]

In [49]:
from scipy.sparse import csr_matrix

In [67]:
mymethod = ('IPCC 2013', 'climate change', 'GWP 100a')

#this was the original code 

l_res = []

# we iterate 10 times
for x in range(0, 10): 
    
    LCA = bw.LCA(FU[0], mymethod)
    LCA.lci()
    LCA.lcia()
    
    # retrieve the coordinates of transforming activities
    transf_col = [LCA.activity_dict[a] for a in transf_act]
    
    # retrieve a sliced copy of the A matrix
    # convert it to a numpy array
    # as it is not possible to manipulate when in sparse format
    arr = LCA.technosphere_matrix[
        np.ix_([c for c in range(0, LCA.technosphere_matrix.shape[0]) if c not in transf_col],
               transf_col)].toarray()
    
    # generate random coefficients the size of the second dimension of `arr`
    # as we want the random coeffecient to apply column-wise (the same coeff for all inputs of a given column)
    coeff = np.random.random_sample((arr.shape[0]))
    
    # multiple the array by the random coefficients
    arr *= coeff.reshape(-1,1)
    
    # convert the array back as a sparse matrix
    arr = csr_matrix(arr)
    
    # update A matrix inputs with new numbers
    LCA.technosphere_matrix[
        np.ix_([c for c in range(0, LCA.technosphere_matrix.shape[0]) if c not in transf_col],
               transf_col)] = arr
    
    print(x)
    
    for j, fu in enumerate(FU):
        LCA.redo_lci(fu)
        LCA.lcia()
        l_res.append((list(fu.keys())[0]["name"], LCA.score))

0
1
2
3
4
5
6
7
8
9


In [68]:
[res for res in l_res if "room-connecting" in res[0]]

[('market for room-connecting overflow element, steel, approx. 40 m3/h',
  1.3522922800186536),
 ('market for room-connecting overflow element, steel, approx. 40 m3/h',
  1.0948579325713181),
 ('market for room-connecting overflow element, steel, approx. 40 m3/h',
  0.749543470057513),
 ('market for room-connecting overflow element, steel, approx. 40 m3/h',
  1.0524796797150393),
 ('market for room-connecting overflow element, steel, approx. 40 m3/h',
  0.45649757496317045),
 ('market for room-connecting overflow element, steel, approx. 40 m3/h',
  0.9566997756445961),
 ('market for room-connecting overflow element, steel, approx. 40 m3/h',
  1.1409637823133003),
 ('market for room-connecting overflow element, steel, approx. 40 m3/h',
  0.7241328450725582),
 ('market for room-connecting overflow element, steel, approx. 40 m3/h',
  1.2351348714320818),
 ('market for room-connecting overflow element, steel, approx. 40 m3/h',
  0.6190062473647335)]

In [23]:
#this is where we were trying to change

for i in range(0,2):
    print(i)
    LCA = bw.LCA(FU[0], ('IPCC 2013', 'climate change', 'GWP 100a'))
    LCA.lci()
    LCA.lcia()
    print(LCA.score)
    transf_col = [LCA.activity_dict[a] for a in transf_act]
    print(transf_col[0:3])
    
    matrix_values = LCA.technosphere_matrix[
        np.ix_([c for c in range(0, LCA.technosphere_matrix.shape[0]) if c not in transf_col],
               transf_col)] 
    #print(matrix_values[0:3])
    
    print(matrix_values[1])
    print(matrix_values[1] * random.uniform(0,1))
    print(matrix_values.shape)
    #for i in matrix_values:
     #   matrix_values[i] = value * random.uniform(0,1)
        
    #print(matrix_values[0:3])
        
  

0
5844.426918696024
[15195, 1578, 8875]


(5499, 10963)
1
5844.426918696024
[15195, 1578, 8875]


(5499, 10963)


In [24]:
import random
from random import randrange

In [25]:
random.uniform(0,1)

0.5175669814007158

In [26]:
LCA.technosphere_matrix[0:5]

<5x16462 sparse matrix of type '<class 'numpy.float64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [27]:
np.ix_([c for c in range(0, LCA.technosphere_matrix.shape[0]) if c not in transf_col],
               transf_col)[0]

array([[    4],
       [    6],
       [    9],
       ...,
       [16448],
       [16451],
       [16458]])

In [28]:
print(matrix_values[2])

  (0, 5467)	1.275823137802945e-06
  (0, 3760)	1.2956618320458801e-06
  (0, 7749)	0.00021740766533184797
  (0, 1666)	1.0298266683861357e-09


In [29]:
for i in matrix_values[0:4]:
    for j in i:
        print(j)
        print(j*10)

  (0, 8968)	-0.4000000059604645
  (0, 9279)	-0.4000000059604645
  (0, 8968)	-4.000000059604645
  (0, 9279)	-4.000000059604645


  (0, 5467)	1.275823137802945e-06
  (0, 3760)	1.2956618320458801e-06
  (0, 7749)	0.00021740766533184797
  (0, 1666)	1.0298266683861357e-09
  (0, 5467)	1.275823137802945e-05
  (0, 3760)	1.2956618320458801e-05
  (0, 7749)	0.0021740766533184797
  (0, 1666)	1.0298266683861357e-08




In [30]:
matrix_values(0,6148)

TypeError: 'csr_matrix' object is not callable

In [31]:
a[1, :].toarray()

NameError: name 'a' is not defined

In [32]:
bbb = matrix_values[0, :].toarray()

In [33]:
print(bbb[2,2])

IndexError: index 2 is out of bounds for axis 0 with size 1

In [54]:
#this is all old stuff, ignore here and below
print(res) 

                                                                                  GWP
Name                                               Efficiency increase (%)           
calcined clay to generic market for supplementa... 42.857143                 0.168285
clear-cutting, primary forest to arable land, p... 42.857143                62.723173
contouring, bronze                                 42.857143                 0.173908
copper production, primary                         42.857143                 2.777568
electricity production, oil                        42.857143                 1.191101
...                                                                               ...
treatment of waste polyethylene, municipal inci... 42.857143                -2.990626
treatment of waste polystyrene, open dump, mois... 42.857143                -0.150363
water pump operation, diesel                       42.857143                 0.083285
white asparagus production                         42.

In [12]:
random.choice(l_res) #how can you select fx 50 items from this list?

(42.85714285714284,
 'treatment of waste polyethylene, municipal incineration',
 -2.990626113935787)

In [13]:
random.sample(l_res, 50) #taking a random sample of 50 exchanges. 

[(233.33333333333317,
  'laser machining, metal, with CO2-laser, 2700W power',
  0.7318252079855012),
 (399.99999999999955,
  'primary zinc production from concentrate',
  0.3587326692347461),
 (233.33333333333317,
  'primary zinc production from concentrate',
  0.5246377611356899),
 (149.99999999999991, 'orbencarb production', 3.3694353884027546),
 (233.33333333333317,
  'market for electricity, low voltage',
  0.04824485242772918),
 (25.0,
  'electricity voltage transformation from high to medium voltage',
  0.04360740335045067),
 (0.0, 'white asparagus production', 1.8753142431840586),
 (99.99999999999996,
  'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted',
  0.0018236119730362904),
 (0.0, 'treatment of decommissioned road', -0.03354304313437791),
 (25.0,
  'electricity voltage transformation from high to medium voltage',
  0.10795453173352149),
 (25.0,
  'market for construction work, heat and power co-generation unit, 160kW electrica

In [14]:
rand = random.sample(l_res, 50)

In [15]:
acts = pd.DataFrame(rand, columns=['Efficiency increase (%)', 'Name', 'GWP'])\
    .groupby(["Name", "Efficiency increase (%)"])[["GWP"]].sum()

In [16]:
print(acts)

                                                                                  GWP
Name                                               Efficiency increase (%)           
calcined clay to generic market for supplementa... 66.666667                 0.136502
clear-cutting, primary forest to arable land, p... 100.000000               62.618066
                                                   400.000000               62.481332
contouring, bronze                                 11.111111                 0.350493
copper production, primary                         150.000000                1.349778
...                                                                               ...
treatment of waste polystyrene, open dump, mois... 66.666667                -0.150363
                                                   233.333333               -0.150363
zinc sulfide production                            0.000000                  5.291893
                                                   66.

In [43]:
acts.to_excel(r'C:\test\Book2.xlsx') #Ok this kind of works but isnt taking 50 FUs, just 50 exchanges. 